In [1]:
import pandas as pd
import numpy as np

In [2]:
from collections import Counter

In [3]:
from utils import EvalMetric

Using TensorFlow backend.


In [4]:
import matplotlib.pyplot as plt 
# import seaborn as sns
import pandas as pd
# sns.set(color_codes=True)
import cufflinks as cf
cf.go_offline()
cf.set_config_file(theme='ggplot')

In [5]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.tools as tls
import plotly.graph_objs as go
import plotly.figure_factory as ff
init_notebook_mode(connected=True)

In [6]:
df = pd.read_pickle('result/prediction.rand_seqs_var_len_sample_ensemble_10_100_100.2019_09_12_1.sample100.pkl')

In [7]:
df.head()

,one_idx,pred_idx,seq
0,"[([2, 3, 4], [17, 16, 15]), ([1, 2, 3, 4, 6], ...","[([1, 2, 3, 4, 6], [18, 17, 16, 15, 13]), ([1,...",AACCUAUGUACGAAAAGGU
1,"[([7, 9, 10, 11, 12], [25, 23, 22, 21, 20]), (...","[([7, 9, 10, 11, 12], [25, 23, 22, 21, 20]), (...",GUGACGCUGGCAGUCGUACUCUGCGGAU
2,"[([0, 1, 2, 3, 4, 5, 6, 11, 14, 15, 17, 18, 19...","[([0, 2, 4, 5, 6, 7, 8, 23, 25, 26, 29, 30, 33...",GUUGCGGACAUGCCGAGUCUGGGUCGAAGUUCUACCUCCAAGUACA...
3,"[([4, 5, 7, 10, 11, 12, 13, 21, 22, 23, 24, 51...","[([4, 5, 12, 13, 14, 15, 16, 21, 22, 23, 24, 3...",AAUUGUGUGACCGGUAUUCCUGCACCCAGUGCUUCUACCCGGGAGG...
4,"[([0, 3, 4, 6, 7, 8], [19, 17, 16, 15, 14, 13]...","[([2, 3, 4, 6, 7, 8], [18, 17, 16, 15, 14, 13]...",CGGACGUGCACAGGCAGUUGC


In [8]:
# convert to tuple so we can use set()
def _to_tuple(idxes):
    x = []
    for _a in idxes:
        a = []
        for i, j in zip(_a[0], _a[1]):
            a.append((i, j))
        if len(a) > 0:
            a = tuple(a)
            x.append(a)
    return x

In [9]:
def _to_mat(x, l):
    # x is tuple of tuple (2 elements)
    # convert to list of 2 lists
    y = [[a[0] for a in x], [a[1] for a in x]]
    z = np.zeros((l, l))
    z[y] = 1
    return z

In [10]:
n_no_struct = 0
df_metric = []

for _, row in df.iterrows():
    seq = row['seq']
    rnafold_struct_count = Counter(_to_tuple(row['one_idx']))
    model_struct_count = Counter(_to_tuple(row['pred_idx']))
    # only keep structures that occurs more than 10 times (10% in sampled structures)
    rnafold_struct_count = {k: v for k, v in rnafold_struct_count.iteritems() if v >=10}
    model_struct_count = {k: v for k, v in model_struct_count.iteritems() if v >=10}
    
    # skip the case if RNAfold outputs no structure
    if len(rnafold_struct_count) == 0:
        n_no_struct += 1
        continue
    
    # for each structure from RNAfold
    # evaluate against all structures from the model, pick the closest one, record performance
#     result = []
    for rnafold_struct in rnafold_struct_count.keys():
        r_s = _to_mat(rnafold_struct, len(seq))
        sensitivity = 0
        ppv = 0
        f_measure = 0
        best_m_s = None
        for model_struct in model_struct_count.keys():
            m_s = _to_mat(model_struct, len(seq))
            _sensitivity = EvalMetric.sensitivity(m_s, r_s)
            _ppv = EvalMetric.ppv(m_s, r_s)
            _f_measure = EvalMetric.f_measure(_sensitivity, _ppv)
            if _f_measure > f_measure:
                sensitivity = _sensitivity + 0  # copy
                ppv = _ppv + 0
                f_measure = _f_measure + 0
                best_m_s = list(model_struct)
#         result.append((sensitivity, ppv, f_measure))
        df_metric.append({
            'seq': seq,
            'rnafold_idx': list(rnafold_struct),
            'closest_model_idx': best_m_s,
            'sensitivity': sensitivity,
            'ppv': ppv,
            'f_measure': f_measure,
        })
    
#     print rnafold_struct_count
#     print model_struct_count
#     print result
#     print ''
    
    
print n_no_struct
df_metric = pd.DataFrame(df_metric)


/Users/alicegao/anaconda2/envs/dg_work_py2/lib/python2.7/site-packages/ipykernel_launcher.py:6: FutureWarning:

Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.

utils.py:325: RuntimeWarning:

invalid value encountered in double_scalars



40


In [11]:
fig = df_metric[['sensitivity', 'ppv','f_measure']].iplot(kind='histogram', bins=20,
                                                    histnorm='probability',
                                                     barmode='group', asFigure=True)
fig = fig.update(layout=dict(
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
    xaxis=dict(showgrid=False, title='Performance Metric', zeroline=False),
    yaxis=dict(showgrid=False, title='Percentage'),
    title='Performance on test dataset',
))

In [12]:
iplot(fig)

In [13]:
plot(fig)

'file:///Users/alicegao/work/psi-lab-sandbox/rna_ss/eval/rnafold_mini_data_2d_ar_2/temp-plot.html'

In [14]:
len(df_metric)

112